In [15]:
import cobra
import pandas as pd
import os

base_path = os.getcwd()

model = cobra.io.load_json_model(os.path.join(base_path, "e_coli_core.json"))
expr_df = pd.read_csv(os.path.join(base_path, "e_coli_core_expression.csv"))
expr_df

,# Reaction ID,reaction activity [mmol/gDW/h]
0,PFK,12.12
1,PFL,1.00
2,PGI,13.12
3,PGK,23.13
4,PGL,8.12
...,...,...
68,ME2,3.34
69,NADH16,20.26
70,NADTRHD,1.26
71,NH4t,3.45


In [17]:
expr_dict = dict(zip(expr_df["# Reaction ID"], expr_df[" reaction activity [mmol/gDW/h] "]))

# Constraints
for rxn in model.reactions:
    if rxn.id in expr_dict:
        value = expr_dict[rxn.id]

        if rxn.id == "EX_glc__D_e":  # glucose exchange → leave wide bounds
            continue

        if rxn.id == "ATPM":  # ATP maintenance → keep lower bound
            rxn.upper_bound = value
            continue

        if rxn.lower_bound < 0:  # reversible
            rxn.lower_bound = -value
            rxn.upper_bound = value
        else:  # irreversible
            rxn.upper_bound = value

# Collect bounds into a DataFrame
bounds_table = pd.DataFrame(
    [(rxn.id, rxn.lower_bound, rxn.upper_bound) for rxn in model.reactions],
    columns=["Reaction", "Lower bound", "Upper bound"]
)

print(bounds_table.to_string(index=False))

                Reaction  Lower bound  Upper bound
                     PFK         0.00        12.12
                     PFL         0.00         1.00
                     PGI       -13.12        13.12
                     PGK       -23.13        23.13
                     PGL         0.00         8.12
                   ACALD        -1.16         1.16
                  AKGt2r        -3.10         3.10
                     PGM       -20.01        20.01
                   PIt2r        -6.03         6.03
                  ALCD2x        -9.01         9.01
                  ACALDt        -2.29         2.29
                    ACKr        -1.19         1.19
                     PPC         0.00         2.56
                  ACONTa       -25.35        25.35
                  ACONTb       -25.35        25.35
                    ATPM         8.39      1000.00
                    PPCK         0.00        25.23
                   ACt2r        -3.23         3.23
                     PPS       

In [19]:
cobra.io.save_json_model(model, "e_coli_core_constrained.json") #saving the model for next exercises